# Szyfry strumieniowe oparte o rejestry przesuwające z sprzeżeniem zwrotnym

Rejestr przesuwający to każda konstrukcja składająca sie z uporządkowanych liniowo komórek pamięci, w której w takt pracy zegara zawartości komórek pamięci jest kopiowana synchronicznie do najbliższych sąsiadów. Rejestr przesuwający może być zrealizowany programowo (nawet na tablicy), ale najczęściej implementuje się je sprzętowo przy użyciu przerzutników (https://pl.wikipedia.org/wiki/Przerzutnik).
Ponieważ w każdym cyklu zegara zawartości komórek przesuwają się w prawo (lub w lewo) to pierwsza komórka pamięci z lewej strony (lub z prawej) opróżnia się. Aby zapewnić poprawną pracę rejestru należy dostarczać do tej komórki odpowiednią zawartość. Można to robić przy pomocy sprzężenia zwrotnego czyli użycia kilku lub wszystkich komórek pamięci oraz pewnej funkcji do wyliczenia wartości wprowadzanej do rejestru.
Rejestry ze sprzężeniem zwrotnym (_feedback shifting registers_) posiadają zdolność do produkowania chaotycznie wyglądających sekwencji danych.

![fsr-stream.svg](attachment:fsr-stream.svg)

# Wprowadzenie

Linear-feedback shift register (LFSR)  
Rejestr przesuwający z liniowym sprzężeniem zwrotnym  
Rejestr przesuwający, którego bit wejściowy jest funkcją liniową jego poprzedniego stanu  

Na zdjęciach poniżej można zobaczyć jak dla 5bitowego rejestru xorowane są 4 wartości w nową wartość, która staje się bitem początkowym.

![State1](assets/l1.png "State 1")
![State2](assets/l2.png "State 2")

Najważniejszym elementem w LFSR są wielomiany pierwotne oraz operacja XOR. Operacja XOR umożliwa rozkład zer i jedynek porównywalny do poziomu 50%. Wielomiany pierwotne zgodnie z algebrą będą generowały najdłuższe możliwe okresy przez co generowany strumień będzie bardziej losowy.

Przykładowa inicjalizacja obiektu, z przejściem do kolejnego stanu.
Należy zdefiniować stan początkowy dla rejestru oraz które wartości rejestru będą uwzględniane w sprzężeniu zwrotnym (_Feedback polynomial_)
Przy inicjalizacji _Output bit_ i _Feedback bit_ nie istnieją dlatego mają wartość -1.

Obiekt rejestru przechowuje wszystkie potrzebne informacje. Między innymi o bicie wychodzącym, bicie sprzężającym i aktualnym stanie rejestru.

Instalacja biblioteki:
```bash
pip install pylfsr
```


In [ ]:
!pip install pylfsr

In [ ]:
from pylfsr import LFSR

state=[1,1,1,0,1]
polynomial=[5,4,2,1]
L=LFSR(initstate=state, fpoly=polynomial, verbose=True)

L.info()
L.next()


L.runKCycle(10)
L.runFullCycle()



Rejestr przesuwny stosuje się w celu genereowania pseudolosowych liczb.   

Interesuje nas okresowość rejestrów. Najlepiej jak jest maksymalny.   
Maksymalny okres dla danego rejestru to taki, gdzie stowarzyszony z nim wielomian jest wielomianem pierwotnym.   
W odniesieniu do rejestru przesuwnego ten _wielomian_ wskazuje które bity należy ze sobą XOR-ować, żeby uzyskać _feedback bit_.  
Nie ważne jaki będzie stan inicjalizacyjny bo wielomian pierowtny sprawi, że rejestr przejdzie przez wszystkie możliwe stany.  

Ilość wielomianów pierwotnych dla zadanej długości można obliczyć z funkcji Eulera.   
Lub wyciągnąć ją z odpowiedniej funkcji w bibliotece :)    
Natomiast w internecie można znaleźć rozpisaną listę wielomianów pierwotnych do 32 potęgi.  
[partow.net/programming/polynomials](https://www.partow.net/programming/polynomials/index.html)  

Zgodnie z algebrą takie wielomiany gwarantują maksymalny okres przez co są najlepszym rozwiązaniem w rejestrach przesuwnych.  
Funkcja _get_fpolyList()_ zawiera listę wielomianów.


In [ ]:
# Lista wszystkich możliwych wielomianów pierwotnych do 32 potęgi

fpolyDict = L.get_fpolyList()
print(fpolyDict)

# Lista możliwych wielomianów pierwotnych dla danej potęgi
fpolys = L.get_fpolyList(m=7)
print(fpolys)

Jak działa rejestr przesuwny w akcji. Poniżej został zdefiniowany prosty 4 bitowy rejestr.  
Każdy stan jest wypisywany razem z aktualnym bitem wyjściowym oraz zwrotnym. _Feedback bit_ wchodzi na początek rejestru a _Output bit_ opuszcza go.  
Na sam koniec jest sprawdzona proporcja zer i jedynek. Jest bliska 50% - tak jak powinno być.  

In [ ]:
state=[1,1,1,0]
# Sprawdzenie wielomianów pierwotnych dla m=4
# print(L.get_fpolyList(m=4)) # [4,1]

polynomial=[4,1]
L=LFSR(initstate=state, fpoly=polynomial, verbose=True)

out=''

print(f'feedback bit {L.feedbackbit} | ', end='')
for _ in range(2**(len(state)) - 1):
    L.next()
    print(f'feedback bit {L.feedbackbit} | ', end='')
    out += ''.join(str(L.outbit))

print('S: ', L.state) # Powrót do pierwszego stanu po przejściu całego okresu

print('output bits', out)

print('number of 0s: ', out.count('0'))
print('number of 1s: ', out.count('1'))

Narzędzie umożliwia również wizualizację każdego stanu.  Poniżej można wygenerować każdy stan i zobaczyc jak się zmieniał.

In [ ]:
L=LFSR(initstate=[1,1,1,0], fpoly=[4,1])

for _ in range(15):
  L.Viz(title='R1')
  L.next()


Sposób wizualizacji każdego stanu proponowany w [dokumentacji](https://lfsr.readthedocs.io/en/latest/LFSR_Examples.html#example-7-1-visualize-3-bit-lfsr-at-each-step-with-default-counter-start-zero-true)

In [ ]:
L=LFSR(initstate=[1,1,1,0], fpoly=[4,1])
print('count \t state \t\toutbit \t seq')
print('-'*50)
for _ in range(15):
    print(L.count,L.state,'',L.outbit,L.seq,sep='\t')
    L.next()
print('-'*50)
print('Output: ',L.seq)

## Testowanie wielomianów pierwotnych
Funkcja _test_properties()_ sprawdzi dla danego stanu rejestru i zaproponowanego wielomianiu czy generuje on dobry losowy ciąg. Pierwszym, historycznym podejściem do sformułowania metod oceny PRNG były postulaty Golomba:
1) W cyklu sN (sekwencja okresowa) liczba jedynek różni się od liczby zer o co najwyżej jeden
2) W cyklu sN przynajmniej połowa podsekwencji złożona z tych samych bitów ma długość 1, 1/4 długość 2, 1/8 długość 3, itd. aż przekroczymy długość sekwencji; wśród tych podsekwecji powinno prawie tyle samo złożonych z zer i z jedynek
3) Autokorelacja mierzy jest miarą podobieństwa między sekwencją $s$ i jej przesunięciem o $t$ pozycji. Funkcja autokorelacji powinna przyąć tylko dwie wartości: $N$, jeśli przesunięcie wynosi 0 oraz $K$, dla każdego innego przesunięcia, gdzie K jest pewną liczbą całkowitą.

$C(t) = \frac{1}{N} \sum_{n=0}^{N-1} a_n \oplus a_{n+t}$

Sekwencję spełniającą postulaty Golomba nazywamy sekwencją $pn$ (ang, pseudo-noise
seąuence). Postulaty Gołomba są przykładem warunków jaki musi spełnić sekwencja, aby została uznana za losową.

In [ ]:
L=LFSR(initstate=[1,1,1,0], fpoly=[4,2]) # Zły
result  = L.test_properties(verbose=2)

L=LFSR(initstate=[1,1,1,0], fpoly=[4,1]) # Dobry
result  = L.test_properties(verbose=2)

## Problem
Zaimplementuj szyfry strumieniowy binarnie addytywny o długości okresu 1024. Przetestuj poprawność jego działania.

Źródła:
1. https://lfsr.readthedocs.io/en/latest/LFSR_Examples.html#installation
1. https://www.partow.net/programming/polynomials/index.html
1. https://pl.wikipedia.org/wiki/Rejestr_przesuwaj%C4%85cy_z_liniowym_sprz%C4%99%C5%BCeniem_zwrotnym
1. https://pl.wikipedia.org/wiki/Szyfr_strumieniowy